# pandasを用いたRNA-seqデータの解析

## 1. 本講義の目的
本講義では, はじめに基本的なpandasの操作方法を学びます。さらに, 実際のRNA-seqカウントデータについて, 以下のような処理をpandasを用いて行います。<br>
- データの読み込み
- 遺伝子のアノテーション(gene idに対応したdescription)
- カウントデータの正規化（RPM/FPM, FPKM, TPM)
- 発現変動遺伝子の抽出
- サンプル間のクラスタリング　

## 2. pandasの基本操作
  
pandas はデータ解析用のライブラリで, おもに二次元の表形式データ (データフレーム) を主な対象としています。
pandas は, 行列・多次元配列を扱う数値計算ライブラリ numpy を内部的に利用して高速な計算が可能です。また, pandas は時系列データや文字データなどを含んだ様々な表形式ファイルの扱いに長けています。本講義では（時間の関係で）pandas の使い方すべてを網羅することができないので, 詳細は以下をご参照ください。<br><br>
<b>参考</b>: pandas公式サイト"[10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html)" <br>
<b>参考図書</b>: Pythonによるデータ分析入門~Numpy,pandasを使ったデータ処理~（Wes Mckinney著, 小林儀匡ら訳, オライリージャパン, ISBN978-4-87311-655-6）<br>

### 2-1. 準備
まず, pandas を`pd`としてインポートします。同様に numpy を`np`としてインポートします。

In [45]:
# pandasのimport
import pandas as pd
import numpy as np

### 2-2. シリーズ
一次元のデータを扱うためのデータオブジェクトです。<br>
実際の解析では, シリーズを新規に作成する機会は少ないと思われますが, 表形式データの行や列を抽出した場合には, シリーズとして結果が返されることがあります。

#### 2-2-1. シリーズの作成と四則演算
リストからシリーズを作成します。

In [ ]:
L1 = [1, 3, 5, 6, 8]
s1 = pd.Series(L1)

上記の場合, データ形式（dtype）はint64です。

In [46]:
s1

0    1
1    3
2    5
3    6
4    8
dtype: int64

In [47]:
L2 = [1, 3, 5, 6, 8.0] # 小数が含まれている
s2 = pd.Series(L2)
s2  # dtypeはfloat64

0    1.0
1    3.0
2    5.0
3    6.0
4    8.0
dtype: float64

In [48]:
L3 = [1,3,5,6,8.0, "hello"] # 文字列が含まれている
s3 = pd.Series(L3)
s3  # dtypeはobject

0        1
1        3
2        5
3        6
4        8
5    hello
dtype: object

シリーズはデータとインデックス（データのラベル）からできています。<br>
デフォルトでは0から始まる数字ですが,任意のインデックスをつけることもできます。

In [49]:
s4 = pd.Series([100, 200, 300, 400 , 500], index=["geneA", "geneB", "geneC", "geneD", "geneE"])
s4

geneA    100
geneB    200
geneC    300
geneD    400
geneE    500
dtype: int64

#### 2-2-2. Seriesの計算
四則計算は各要素全てが計算対象となります。

In [50]:
s5 = pd.Series([10,30,50,60,80,100])

In [51]:
print(s5 + 10)

0     20
1     40
2     60
3     70
4     90
5    110
dtype: int64


In [52]:
print(s5 / 10) 

0     1.0
1     3.0
2     5.0
3     6.0
4     8.0
5    10.0
dtype: float64


python3では / の結果は `float` (小数点のついた値)で返ります。// を使うと `int` （整数値）で返ります。<br>
i.e. `10/2=5.0  10//2=5  9//2=4` <br>

シリーズ同士の掛け算では, 同じ位置にある要素同士が計算されます。

In [10]:
s1 * s5

0     10.0
1     90.0
2    250.0
3    360.0
4    640.0
5      NaN
dtype: float64

#### 2-2-3. 位置を指定したデータの切り出し

In [55]:
s6 = pd.Series(list("ABCDEFGH"))

位置を指定して単独の要素を取得することができます。

In [56]:
print(s6[3])

D


リストと同じようにスライスを使ってデータを切り出すことができます。<br>
もとのデータ型が `object` なので切り出しても `object` のままであることに注意しましょう。

In [57]:
print(s6[2:5])  # 結果はシリーズとして返されます

2    C
3    D
4    E
dtype: object


In [58]:
print(s6[:-1])

0    A
1    B
2    C
3    D
4    E
5    F
6    G
dtype: object


複数の位置を指定することもできます。

In [59]:
targets = [0, 2, 4, 6]
s6[targets]

0    A
2    C
4    E
6    G
dtype: object

上記を一行で書くときは, 次のように`[ ]`が二重` [ [ ] ] `になります。`s1[0, 2, 4, 6, 8]` とするとエラーになります。<br>
二重の`[ ]`は pandas では頻繁に登場します。

In [60]:
s6[[0, 2, 4, 6]]

0    A
2    C
4    E
6    G
dtype: object

以下の違いに注意してください。

In [61]:
print("s6[0]:")  # 単独の数値で指定
print(s6[0])
print(type(s6[0]))
print("------------------")
print("s6[[0]]:")  # リストで指定, Seriesで返る
print(s6[[0]])
print(type(s6[[0]]))     

s6[0]:
A
<class 'str'>
------------------
s6[[0]]:
0    A
dtype: object
<class 'pandas.core.series.Series'>


シリーズをリストにするには `list()` を使います。

In [62]:
list(s6)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

#### 2-2-4. インデックスを使ったデータの切り出し
インデックスを指定してデータを取り出します。

In [63]:
s4

geneA    100
geneB    200
geneC    300
geneD    400
geneE    500
dtype: int64

In [64]:
s4["geneA"]

100

辞書を使ってインデックスを指定したシリーズを作成することもできます。

In [66]:
D = {'Chlamydomonas': 'reinhardtii', 'Volvox': 'carteri', 'Coccomyxa': 'subellipsoidea', 'Coffea': ' arabica'}
s7 = pd.Series(D)

インデックスが Chlamydomonas, Volvox である値を取り出します。

In [67]:
s7[['Chlamydomonas', 'Volvox']]

Chlamydomonas    reinhardtii
Volvox               carteri
dtype: object

インデックスに用いる値はユニークでなくても（重複していても）インデックスとして使えます。

In [68]:
data= [ 'reinhardtii', 'carteri',  'subellipsoidea_C_169', ' arabica', 'canephora']
index = ['Chlamydomonas',  'Volvox', 'Coccomyxa', 'Coffea', 'Coffea']
s8 = pd.Series(data=data, index=index)
print(s8)

Chlamydomonas             reinhardtii
Volvox                        carteri
Coccomyxa        subellipsoidea_C_169
Coffea                        arabica
Coffea                      canephora
dtype: object


ただし,以下のように異なったデータ形式で結果が返ってくることがあるため, インデックスに重複した値を使うのは避けたほうがよいかもしれません。

In [69]:
s8['Coffea']  # ２つの要素が返る (シリーズとして返る)

Coffea      arabica
Coffea    canephora
dtype: object

In [70]:
s8['Chlamydomonas']  # 単独の要素が返る (str型)

'reinhardtii'

#### 2-2-5. インデックスおよびデータの参照
`Series.index`, `Serires.values`でそれぞれインデックスとデータを取り出せます。<br>
新しい値を指定することも可能です。

In [71]:
s8.index

Index(['Chlamydomonas', 'Volvox', 'Coccomyxa', 'Coffea', 'Coffea'], dtype='object')

In [72]:
s8.values

array(['reinhardtii', 'carteri', 'subellipsoidea_C_169', ' arabica',
       'canephora'], dtype=object)

データ形式を確認してみると, 内部的に numpy が使われていることがわかります。

In [73]:
type(s8.values) 

numpy.ndarray

### 2-3. データフレーム 
二次元の表を扱うためのデータオブジェクトです。<br>
#### 2-3-1. データフレームの作成
`pd.DataFrame( )`でデータフレームを作成します。引数のうち `data` には、二次元リストや辞書などを指定します。`index`（行ラベル）, `columns`（列ラベル）は指定しなかった場合, 0, 1, 2, .. が自動で割り振られます。

In [74]:
data = [[1, 3, 2],
        [10, 20, 30],
        [100, 200, 300],
        [1000, 2000, 3000]
       ]
columns = ['sample1', 'sample2', 'sample3']
df1 = pd.DataFrame(data, columns=columns)
df1

,sample1,sample2,sample3
0,1,3,2
1,10,20,30
2,100,200,300
3,1000,2000,3000


上の表はデフォルトのインデックス（行ラベル, 0から始まる整数値）ですが, 後から任意のインデックスをつけることができます。

In [75]:
df1.index = ['geneA','geneB','geneC','geneD']
df1

,sample1,sample2,sample3
geneA,1,3,2
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


辞書データを与える場合は以下のようになります。列ごとにデータを指定します。

In [76]:
D = {'generic_name': ['Chlamydomonas', 'Volvox','Ostreococcus' ,'Coccomyxa', 'Marchantia','Thalassiosira'],
     'specific_name': ['reinhardtii', 'carteri','lucimarinus', 'subellipsoidea C-169', 'polymorpha','pseudonana'],     
     'assemble_version': ['v5.5','v2.1','v2.0','v2.0','v3.1','V3.0'],
     'genome_size': [111.1,131.2,13.2,49,225.8,34],
     'common_name':['green algae','green algae','green algae','green algae','liverwort','diatom'],
     'unicellular': [True,False,True,True,False,True]
     }

df2 = pd.DataFrame(D)
df2

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True


カラム名（列ラベル）や, インデックス（行ラベル）はそれぞれ、`columns`, `index` で参照できます。

In [77]:
print("column names: ", df2.columns)
print("index: ", df2.index)

column names:  Index(['generic_name', 'specific_name', 'assemble_version', 'genome_size',
       'common_name', 'unicellular'],
      dtype='object')
index:  RangeIndex(start=0, stop=6, step=1)


RNA-seq などの解析においては, ファイルから読み込んでデータフレームを作成する場合がほとんどだと思います。<br>
ファイルから読み込む時は `pd.read_table()` や `pd.read_csv()` を使います。これ以外にも `pd.read_clipboard`, `pd.read_excel` などもあります。<br>
このあと, 2-6 でタブ区切りのファイルを読み込む方法を紹介します。

####  2-3-2. データフレームを使った計算
データフレームを使った計算をしてみましょう。<br>

In [78]:
df1

,sample1,sample2,sample3
geneA,1,3,2
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


スカラー値との四則演算は各要素に対して行われます。

In [79]:
df1 * 10

,sample1,sample2,sample3
geneA,10,30,20
geneB,100,200,300
geneC,1000,2000,3000
geneD,10000,20000,30000


上記の計算はもとのデータが変更されるのではなく, 新しいデータフレームとして結果が返されます。<br>
そのため、データの値を更新したい場合は, `df = df * 10` のように結果を元のデータフレームに代入する必要があります。

リストやシリーズとの計算もできます。

In [81]:
L = [1, 2, 3]
df1 + L

,sample1,sample2,sample3
geneA,2,5,5
geneB,11,22,33
geneC,101,202,303
geneD,1001,2002,3003


各行ごとに計算が適用されます。（ブロードキャスト）<br>
下の例ではデータの要素数が揃っていないのでエラーになります。

```
L = [1, 2, 3, 4]
df1 + L
```

Q. 横方向にブロードキャストするにはどうするか？ <br>
  
A. `df.T` を使い行と列を転置させてから計算し, その後, ふたたび転置させてもとの形に戻します。

In [82]:
L = [1, 2, 3, 4]
(df1.T + L).T

,sample1,sample2,sample3
geneA,2,4,3
geneB,12,22,32
geneC,103,203,303
geneD,1004,2004,3004


データフレーム同士の計算も可能です。<br> 
データフレームの形状が同じであれば要素ごとに計算できますが, あまり利用機会はないと思いますので省略します。 <br> 
また, 行列の内積のような計算も可能ですが, 本格的な行列の数値計算であれば numpy を使った方が便利です。

__集計関数__<br>

列ごとの合計は `sum()` を使います。デフォルトは列ごとの合計 `（axis=0）` です。行ごとの合計を求める時は `axis=1` とします。 

In [83]:
df1.sum()  # 列ごとの合計 (axis=0)

sample1    1111
sample2    2223
sample3    3332
dtype: int64

In [84]:
df1.sum(axis=1) # 行ごとの合計

geneA       6
geneB      60
geneC     600
geneD    6000
dtype: int64

In [85]:
df1.mean() # 平均

sample1    277.75
sample2    555.75
sample3    833.00
dtype: float64

In [86]:
df1.cumsum(axis=0) # 累積 (上で紹介した他の集計関数と異なり, 結果はデータフレームとして返る)

,sample1,sample2,sample3
geneA,1,3,2
geneB,11,23,32
geneC,111,223,332
geneD,1111,2223,3332


__インデックスでソート__ `sort_index()`

`ascending = False` とすると降順でソートされます。(デフォルトは昇順)

In [87]:
df1.sort_index(ascending=False).head()

,sample1,sample2,sample3
geneD,1000,2000,3000
geneC,100,200,300
geneB,10,20,30
geneA,1,3,2


__要素の値でのソート__ `sort_values()`  

In [88]:
df2

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True


genome_size 列の値でソートします。

In [90]:
df2.sort_values(by='genome_size', na_position='first')

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False


`inplace=True` オプションをつけると, もとのデータフレームを書き換えます（破壊的変更）。
generic_name 列でソートします。

In [91]:
df2.sort_values(by='generic_name', axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')

順番が変わっていることを確認しましょう。

In [92]:
df2

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True
1,Volvox,carteri,v2.1,131.2,green algae,False


インデックス順にソートして元に戻しておきます（`inplace=True` オプション)。<br>
なお, df2のインデックスはデフォルト(0~5の整数) です。  

In [93]:
df2.sort_index(ascending=True, inplace=True)

__列の並び替え__ (あまり使用する機会はないかもしれませんが)

In [94]:
df1.head(2)

,sample1,sample2,sample3
geneA,1,3,2
geneB,10,20,30


geneAの行の値は左から 1 → 3 → 2 と並んでいますが, これを左から昇順( 1 → 2 → 3 )に並べ替えます。

In [95]:
df1.sort_values(by="geneA", axis=1).head(2) # axis=1を指定

,sample1,sample3,sample2
geneA,1,2,3
geneB,10,30,20


`inplace=True` オプションをつけていないのでdf1自体は変更しません。

### 2-4. データフレームから行・列・要素の抽出
#### 2-4-1. 行・列の抽出

In [96]:
df1

,sample1,sample2,sample3
geneA,1,3,2
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


__列の抽出__<br>

抽出したい列名を`[ ]`で指定します。結果はシリーズとして返ります。

In [97]:
df1['sample1'] 

geneA       1
geneB      10
geneC     100
geneD    1000
Name: sample1, dtype: int64

以下のやりかたでも同様の操作が可能です。

In [98]:
df1.sample1 

geneA       1
geneB      10
geneC     100
geneD    1000
Name: sample1, dtype: int64

複数の列を抽出する場合にはリストで指定します。`[ ]`が二重`[[ ]]`になることに注意してください。<br>
結果はデータフレームとして返ります。

In [99]:
df1[['sample1','sample3']]  

,sample1,sample3
geneA,1,2
geneB,10,30
geneC,100,300
geneD,1000,3000


sample1列のみをデータフレームとして抽出します。最初の例 ` df1['sample1']` との違いに注意しましょう。

In [100]:
df1[['sample1']] 

,sample1
geneA,1
geneB,10
geneC,100
geneD,1000


__スライスを使った行の抽出__<br>

行の抽出にスライスを使うことができますが, 後述の `loc` または `iloc` を覚えておけばよいでしょう。

In [101]:
# 0から2行目までを抽出
df2[0:3]

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True


「0」という名称の列が存在しないため、`df3[0]`とするとエラーになります。<br>
スライスで0行目を抽出するときは以下のように指定します。

In [102]:
df2[0:1]

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True


#### 2-4-2. 要素の抽出 ( at, iat )
`at` でインデックスとカラム名（ 行ラベル, 列ラベル ）を指定して要素を抽出します。<br>
[　行, 列　]の順に指定します。

In [103]:
df1

,sample1,sample2,sample3
geneA,1,3,2
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


In [104]:
df1.at['geneB', 'sample2']

20

```iat```　で位置を指定して要素を抽出します ( `i` はintegerの意味 )。

In [48]:
df1.iat[1, 1]

20

#### 2-4-3. 範囲（複数、単数の行および列）の切り出し（ loc, iloc ）
`loc` でインデックスによる抽出ができます。単独の要素の抽出は以下のように書きます。<br>
`df3.at["data_1", "A"]` と同じ結果になりますが、`at` の方が速いです。

In [105]:
df1.loc['geneA', 'sample2'] 

3

複数の行・列からなる範囲の抽出は以下のようにします。結果はデータフレームとして返ります。

In [106]:
df1.loc['geneA':'geneC', 'sample2':'sample3']  

,sample2,sample3
geneA,3,2
geneB,20,30
geneC,200,300


上の例で, 通常の数値によるスライスとは異なり, geneC 行や sample3 列が含まれていることに注意してください。<br>
慣れるまでは, 実際に抽出したデータが自分の欲しいものかどうかを確認しながら進めたほうがよいでしょう。

':' を指定すると対象となる列 or 行の全体が抽出されます。以下の例では geneB ~ geneD までのすべての列を表示します。

In [107]:
df1.loc['geneB':'geneD', :] 

,sample1,sample2,sample3
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


列の指定を省略することも可能です。行の全体を抽出するにはこの方法が便利です。

In [108]:
df1.loc['geneB':'geneD']

,sample1,sample2,sample3
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


以下は ':' ですべての行を指定する例です。

In [109]:
df1.loc[:, 'sample1':'sample3']

,sample1,sample2,sample3
geneA,1,3,2
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


ただし, すべての行を指定する場合, `df.loc[:,['A', 'B', 'C']]`より`df[['A', 'B', 'C']]`の書き方をおすすめします。

リストを使って行や列を指定することも可能です。この方法を使って任意の並び順にすることができます。

In [110]:
df1.loc[['geneB','geneC','geneA'], ['sample2','sample1']]  

,sample2,sample1
geneB,20,10
geneC,200,100
geneA,3,1


すべての列を取得する場合は以下のようにします。`df3.loc[['geneC','geneA], :]`でも同じ結果になります。

In [111]:
df1.loc[['geneC','geneA']]  

,sample1,sample2,sample3
geneC,100,200,300
geneA,1,3,2


すべての行を取得する場合は以下のようにします。

In [112]:
df1.loc[:, ['sample3','sample1']] 

,sample3,sample1
geneA,2,1
geneB,30,10
geneC,300,100
geneD,3000,1000


繰り返しになりますが, 上の例より `df1[['sample3','sample1']]` の方が便利です。

単独の行の抽出は以下のようにします。

In [113]:
df1.loc['geneD', :]

sample1    1000
sample2    2000
sample3    3000
Name: geneD, dtype: int64

列の指定 `:,` は省略できます。

In [114]:
df1.loc['geneD']

sample1    1000
sample2    2000
sample3    3000
Name: geneD, dtype: int64

単独の列の抽出は以下のようにします。

In [115]:
df1.loc[:, 'sample2']

geneA       3
geneB      20
geneC     200
geneD    2000
Name: sample2, dtype: int64

行の指定 `:,` は省略できます。

In [116]:
df1['sample2']

geneA       3
geneB      20
geneC     200
geneD    2000
Name: sample2, dtype: int64

`[[ ]]`で指定した時と`[ ]`で指定した時、返ってくるデータの形式が異なることに注意しましょう。

In [117]:
df1.loc[['geneD']]  # 結果はデータフレーム

,sample1,sample2,sample3
geneD,1000,2000,3000


In [118]:
df1.loc['geneD']  # 結果はシリーズ

sample1    1000
sample2    2000
sample3    3000
Name: geneD, dtype: int64

`iloc` で位置（整数値）による指定ができます。位置を数字で指定することを除いて `loc` と同じです。<br>

[ 行, 列 ] と指定します。

In [119]:
df1.iloc[1,2]

30

スライスで指定することもできます。

In [120]:
df1.iloc[1:2, 0:2]

,sample1,sample2
geneB,10,20


リストで指定することもできます。

In [121]:
df1.iloc[[1,2], [0,1,2]]

,sample1,sample2,sample3
geneB,10,20,30
geneC,100,200,300


すべての列（行全体）を取得します。

In [122]:
df1.iloc[1,:]

sample1    10
sample2    20
sample3    30
Name: geneB, dtype: int64

列の指定を省略しても同じ結果が得られます。

In [123]:
df1.iloc[1]

sample1    10
sample2    20
sample3    30
Name: geneB, dtype: int64

上の例と同じ行を抽出していますが, リストで指定した場合はデータフレームとして返ります。

In [124]:
df1.iloc[[1]]

,sample1,sample2,sample3
geneB,10,20,30


すべての行（列全体）を取得します。

In [125]:
df1.iloc[:,1]

geneA       3
geneB      20
geneC     200
geneD    2000
Name: sample2, dtype: int64

#### 2-4-4. まとめ
* __列の抽出__  
`df['A']` または `df.A`  
複数列の場合にはリストで指定する（`[ ]` が二重 `[[ ]]` になることに注意）  
`df[['A', 'B']]`


* __行の抽出 ( loc or ilocを使う )__  
`df.loc[row_index]` または `df.iloc[row_position]`  
複数行の場合には、  
`df.loc[[row_index1, row_index2]]`  または  
`df.iloc[[row_position1, row_position2]]`  または  
`df.iloc[row_position1:row_position2]`


* __範囲の抽出 ( loc or ilocを使う )__  
`df.loc[[row_index1, row_index2], [col_index1, col_index2]]`  または  
`df.iloc[[row_position1, row_position2], [col_position1, col_position2]]`  または  
`df.iloc[row_position1:row_position2, col_position1:col_position2]`


* __値の抽出 ( at or iatを使う )__   
`df.at[row_index, col_index]`  または  `df.iat[row_position, col_position]`  
( loc or ilocよりも速い )


#### 2-4-4. 条件を指定して抽出 ( boolean indexing )

抽出対象をbool値（TrueまたはFalse）のリストで指定します。

In [126]:
df2

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True


In [127]:
targets = [True, True, False, False, True, False]
df2[targets]

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False


df2から `unicellular=True` のデータのみを抽出します。

In [128]:
df2[df2.unicellular]

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True


~ をつけると not の意味になります。

In [129]:
df2[~df2.unicellular]

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
1,Volvox,carteri,v2.1,131.2,green algae,False
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False


genome_size 列の値が100以上のものを抽出します。

In [130]:
df2[df2.genome_size >= 100]

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False


common_name 列の値が green algae であるものを抽出します。

In [131]:
df2[df2['common_name'] == 'green algae']

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True


### 2-5. データフレームへ新規の行・列の追加, 行・列の編集
#### 2-5-1. 列の操作
__列の追加__<br>

データフレーム名[ '列ラベル' ] = [ データのリスト ] で新しい列が追加できます。

In [132]:
df1

,sample1,sample2,sample3
geneA,1,3,2
geneB,10,20,30
geneC,100,200,300
geneD,1000,2000,3000


In [133]:
df1['sample4'] = [4, 40, 400, 4000]  # すでにsample4列が存在している場合には上書きされる
# 確認
df1

,sample1,sample2,sample3,sample4
geneA,1,3,2,4
geneB,10,20,30,40
geneC,100,200,300,400
geneD,1000,2000,3000,4000


__列の編集__<br>

sample3列を10倍し, sample4列のデータをすべて入れ替えます。

In [134]:
df1['sample3'] = df1['sample3'] * 10
df1['sample4'] = [4, 44, 444, 4444]
df1

,sample1,sample2,sample3,sample4
geneA,1,3,20,4
geneB,10,20,300,44
geneC,100,200,3000,444
geneD,1000,2000,30000,4444


sample1 ~ sample4 の平均を計算し, AVERAGE 列を追加します。

In [135]:
df1['AVERAGE'] = (df1['sample1'] + df1['sample2'] + df1['sample3'] + df1['sample4']) / 4

上の例は `mean()` 関数を使っても同じ結果が得られます。

In [136]:
df1['AVERAGE2'] = df1[['sample1', 'sample2', 'sample3', 'sample4']].mean(axis=1)
df1

,sample1,sample2,sample3,sample4,AVERAGE,AVERAGE2
geneA,1,3,20,4,7.0,7.0
geneB,10,20,300,44,93.5,93.5
geneC,100,200,3000,444,936.0,936.0
geneD,1000,2000,30000,4444,9361.0,9361.0


__列の削除__<br>

列の削除には `drop()` を使います。デフォルトでは` axis=0` (行)の削除なので, 列の削除では `axis=1`を指定します。

In [137]:
df1.drop('AVERAGE', axis=1) 

,sample1,sample2,sample3,sample4,AVERAGE2
geneA,1,3,20,4,7.0
geneB,10,20,300,44,93.5
geneC,100,200,3000,444,936.0
geneD,1000,2000,30000,4444,9361.0


リストを使って複数列も同時に指定可能です。

In [138]:
df1.drop(['AVERAGE', 'AVERAGE2'], axis=1) 

,sample1,sample2,sample3,sample4
geneA,1,3,20,4
geneB,10,20,300,44
geneC,100,200,3000,444
geneD,1000,2000,30000,4444


`drop`は非破壊的変更なので, 実際には上記の例では削除できていません。<br>
削除するためには, `df = df.drop('AVERAGE', axis=1)`というように代入します。

In [139]:
df1

,sample1,sample2,sample3,sample4,AVERAGE,AVERAGE2
geneA,1,3,20,4,7.0,7.0
geneB,10,20,300,44,93.5,93.5
geneC,100,200,3000,444,936.0,936.0
geneD,1000,2000,30000,4444,9361.0,9361.0


`del` を使っても削除可能です。ただし, 破壊的変更なので代入なしで即座に反映されます。

In [140]:
del df1['AVERAGE'], df1['AVERAGE2'],  df1['sample4']
df1.sample3 = df1.sample3 / 10  # 元の値に戻しておきます。
df1
# del df1[['AVERAGE', 'AVERAGE2', 'sample4']] はエラーになります。

,sample1,sample2,sample3
geneA,1,3,2.0
geneB,10,20,30.0
geneC,100,200,300.0
geneD,1000,2000,3000.0


### 2-5-2. 行の操作

__行の追加__<br> 

実際には一行ずつ追加する機会は多くないと思われます。後述の `concat` で複数のデータフレームを連結する操作の方が多いかもしれません。<br>
追加したい行データをあらかじめシリーズ（以下の例では geneE ）として作成しておきます。<br>
このときシリーズのインデックスには、データフレームの列名（columns）を指定, 行名として `name='geneE'` を指定します。<br>

In [141]:
geneE = pd.Series([9,99,999], index = df1.columns, name = 'geneE')
geneE

sample1      9
sample2     99
sample3    999
Name: geneE, dtype: int64

`append()`を使って行を追加します。<br>

In [142]:
df1 = df1.append(geneE)
df1

,sample1,sample2,sample3
geneA,1,3,2.0
geneB,10,20,30.0
geneC,100,200,300.0
geneD,1000,2000,3000.0
geneE,9,99,999.0


__行の編集__<br>  

`loc` または `iloc` を使って行を指定して, 新しい値を代入します。

In [143]:
df1.iloc[4] = df1.iloc[4]*10
df1 

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0
geneE,90.0,990.0,9990.0


5行め, geneEの値が10倍の値になっていることを確認してください。<br>
`df1` を元の値に戻しておきます。ここでは`loc`を使っているので` [ ] `の中身はインデックス（行ラベル）です。

In [144]:
df1.loc['geneE'] = df1.loc['geneE'] / 10  
df1

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0
geneE,9.0,99.0,999.0


要素の値を指定することももちろん可能です。

In [145]:
df1.loc['geneA', 'sample1'] = 0
df1

,sample1,sample2,sample3
geneA,0.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0
geneE,9.0,99.0,999.0


In [146]:
df1.loc['geneA', 'sample1'] = 1 # 元の値に戻しておく
df1

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0
geneE,9.0,99.0,999.0


__行の削除__<br>

`drop()` を使用します。インデックス（行ラベル）を指定します。

In [147]:
df1 = df1.drop('geneE') 
df1

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0


### 2-6. 欠損値・重複の扱い

テストデータとして, タブ区切りのファイル `test_matrix_data.tsv` を読み込みます。<br>
ファイルの読み込みには `pd.read_table()` または `pd.read_csv()` を使います。

In [148]:
df3 = pd.read_table("input/test_matrix_data.tsv", index_col=0)

`df3 = pd.read_csv("input/test_matrix_data.tsv", sep="\t", index_col=0) ` としても同じ結果になります。<br>

In [149]:
df3.head()

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_12,181,126,55.0,0.5000,NaN,0.0551,0.0000
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082


データの大きさを`shape`で確認します。30行 x 7列のデータになっているはずです。

In [150]:
df3.shape

(30, 7)

__欠損値の削除__<br>

データに欠損値（ NaN ）がある行すべてを削除します。この例では data_12 の E列に NaN があるため削除されます。

In [151]:
df3.dropna().head()

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082
data_14,133,-70,63.0,0.5000,374.867722,0.0420,0.0000


非破壊的変更なので実際に削除するには `df3 = df3.dropna()`とするか, `df3.dropna(inplace=True)`とする必要があります。<br>
`shape` で何行削除されたか確認します。

In [152]:
df3.dropna().shape  # 30行x７列 -> 26行×7列になったので4行削除されたことがわかる

(26, 7)

デフォルトでは欠損値を含む行が削除されます(`axis=0`)。欠損値を含む列を削除するには`axis=1`を指定します。<br>
その他に, 特定の行に欠損値が含まれている場合を削除対象にするオプション`subset`や, 欠損値の数の閾値を指定するオプション`thresh`などもあります。

__欠損値の補完__<br>

欠損値を 0 で補完することができます。

In [153]:
df3.fillna(0).head()

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_12,181,126,55.0,0.5000,0.000000,0.0551,0.0000
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082


引数に辞書やシリーズを指定することで, 各行ごとに異なる値を指定することもできます。<br>
下の例では, C列は 0, E列は 1 で補完されます。

In [154]:
Dic = {"C": 0, "E": 1}
df3.fillna(Dic).head()

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_12,181,126,55.0,0.5000,1.000000,0.0551,0.0000
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082


上記の方法を応用して, 各列の平均で埋めることも可能です。

In [155]:
df3.fillna(df3.mean()).head()

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_12,181,126,55.0,0.5000,314.394296,0.0551,0.0000
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082


__前後の値による補完__<br> 

`interpolate()`を使います。

In [156]:
df3.interpolate().head()

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_12,181,126,55.0,0.5000,520.639262,0.0551,0.0000
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082


__重複の除去__<br>

`drop_duplicates()` を使用します。デフォルトでは行全体の値が重複していた場合に削除します。<br>  
下の例ではD列に重複値がある場合に, 先頭の行のみ残して以後の行を削除しています(`keep="first"`)。 <br>
重複している行すべてを削除することも可能です(`keep=False`)。

In [157]:
df3.head(6)

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_12,181,126,55.0,0.5000,NaN,0.0551,0.0000
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082
data_14,133,-70,63.0,0.5000,374.867722,0.0420,0.0000


D列で 0.5000 が重複しています（data_10,data_12,data_14...）。

In [159]:
df3.drop_duplicates(subset="D").head() # data_10が残る

,A,B,C,D,E,F,G
data_idx,,,,,,,
data_1,58,-27,31.0,0.6028,73.922054,0.0179,-0.0102
data_10,109,-53,56.0,0.5000,575.585790,0.0346,0.0000
data_11,77,-44,33.0,0.6281,144.838806,0.0343,-0.0411
data_13,91,-46,45.0,0.5220,896.439718,0.0044,-0.0082
data_19,6,-2,4.0,0.5453,72.282932,0.0220,-0.0707


__(参考)__  
pandasではデータフレームに対する操作の多くは__非破壊的変更__, つまり自分自身を変更させるのではなく、新しいデータフレームとして結果を返します。  
このメリットの一つに、複数のメソッドを連結して使用できることが挙げられます。（メソッドチェイン）  
例: `df3.dropna().drop_duplicates(subset="D").sort_index().head()`


### 2-7. データフレーム全体、または行・列に対しての関数の適用
#### 2-7-1. データフレームの集計メソッドを使用する方法
2-3-2.で紹介済みなので詳細は割愛します。

In [160]:
df3.sum() 

A    2029.000000
B     101.000000
C     939.000000
D      16.628800
E    8488.645986
F       0.799400
G      -0.561800
dtype: float64

In [161]:
df2.sum()

generic_name        ChlamydomonasVolvoxOstreococcusCoccomyxaMarcha...
specific_name       reinhardtiicarterilucimarinussubellipsoidea C-...
assemble_version                             v5.5v2.1v2.0v2.0v3.1V3.0
genome_size                                                     564.3
common_name         green algaegreen algaegreen algaegreen algaeli...
unicellular                                                         4
dtype: object

#### 2-7-2. numpyの関数を利用する方法
numpy にはユニバーサル関数と呼ばれる行列（データフレーム）の要素ごとに処理を行う関数があります。

In [162]:
df1

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0


常用対数を計算しましょう。他に自然対数 ( log )、log2 などもあります。

In [163]:
np.log10(df1)

,sample1,sample2,sample3
geneA,0.0,0.477121,0.301030
geneB,1.0,1.301030,1.477121
geneC,2.0,2.301030,2.477121
geneD,3.0,3.301030,3.477121


ユニバーサル関数はシリーズにも適用できるので特定の列や行に対して実行することも可能です。<br>
`np.sqrt()`で平方根を計算します。値を更新するには　`df1["sample2"] = np.sqrt(df1["sample2"])` というように代入します。

In [164]:
print(np.sqrt(df1['sample2'])) 

geneA     1.732051
geneB     4.472136
geneC    14.142136
geneD    44.721360
Name: sample2, dtype: float64


`np.round()`で四捨五入を計算します。

In [165]:
df3.iloc[1]

A    109.00000
B    -53.00000
C     56.00000
D      0.50000
E    575.58579
F      0.03460
G      0.00000
Name: data_10, dtype: float64

In [166]:
print(np.round(df3.iloc[1]))

A    109.0
B    -53.0
C     56.0
D      0.0
E    576.0
F      0.0
G      0.0
Name: data_10, dtype: float64


numpy の集計関数も利用可能です。

In [167]:
np.var(df3)

A      2404.698889
B      1877.832222
C       682.442331
D         0.004918
E    197282.083194
F         0.000677
G         0.000699
dtype: float64

行方向に適用するときは `axis=1` とします。

In [168]:
np.var(df1, axis=1)

geneA         0.666667
geneB        66.666667
geneC      6666.666667
geneD    666666.666667
dtype: float64

`np.var()` は平均との差の自乗をデータ数 N で割っていますが、不偏分散（N-1で割る）を求めるには `ddof=1` を指定します。

In [169]:
np.var(df1, ddof=1)

sample1     233840.25
sample2     934992.25
sample3    2105116.00
dtype: float64

pandas.DataFrameの`var()`は デフォルトで `ddof=1` として計算されています。

In [170]:
df1.var()

sample1     233840.25
sample2     934992.25
sample3    2105116.00
dtype: float64

標準偏差を求めるには `np.std()` または `df.std()` を使用します。

#### 2-7-3. map, apply, applymap
関数をデータフレームや行・列に適用するのに用います。  
ただし、numpy やデータフレームのメソッドに定義されているものはそれを使用した方が処理が早いです。  

In [171]:
# テスト用関数: 引数xが文字列であれば小文字に変換、そうでなければ "-" を返す
def my_lower(x):    
    if isinstance(x, str):
        return x.lower()
    else:
        return "-"

シリーズ（データフレームの行・列）の各要素に関数を適用するには `map()` を使います。

In [172]:
df2

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True


In [173]:
df2['generic_name'].map(my_lower)

0    chlamydomonas
1           volvox
2     ostreococcus
3        coccomyxa
4       marchantia
5    thalassiosira
Name: generic_name, dtype: object

データフレームの各要素に関数を適用するには `applymap()` を使います。

In [174]:
df2.applymap(my_lower)

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,chlamydomonas,reinhardtii,v5.5,-,green algae,-
1,volvox,carteri,v2.1,-,green algae,-
2,ostreococcus,lucimarinus,v2.0,-,green algae,-
3,coccomyxa,subellipsoidea c-169,v2.0,-,green algae,-
4,marchantia,polymorpha,v3.1,-,liverwort,-
5,thalassiosira,pseudonana,v3.0,-,diatom,-


より複雑な関数をシリーズ（データフレームの行・列）の各要素に適用するには `pandas.Series` の `apply()` を使います。<br>
`Series.apply(func, convert_dtype=True, args=(), **kwds)`<br>

オプション`args`に引数をタプルとして与えることができます。

In [175]:
# テスト用関数その2: 第一引数 x が数値であれば、小数第 n 位までの概数にする(nは負の値も可)
def my_round(x, n):
    if isinstance(x, int) or isinstance(x, float):
        return round(x, n)
    else:
        return np.NaN

シリーズの各値について、小数第二位までの概数にします。

In [176]:
test_s = pd.Series([3.89, 2.192, 15.3921, 43.903, 390.083, 239.622])

`args` に第二引数以降の引数をタプルとして与えます。要素数１のタプルなので `(2,) `と書きます。

In [177]:
test_s.apply(my_round, args=(2,))  

0      3.89
1      2.19
2     15.39
3     43.90
4    390.08
5    239.62
dtype: float64

データフレームの行または列ごとに関数を適用するには, `pandas.DataFrame` の `apply()` を使います。<br>
`pandas.DataFrame` の `apply()` と `pandas.Series` の `apply()` は使い方が異なるので注意してください。<br> 

`df1.apply(func, axis=0, broadcast=False, raw=False, reduce=None, args=(), **kwds)`

In [178]:
# テスト用集計関数: シリーズ (データフレームの行 or 列)を受け取り, thresholdより値が大きいものの個数を返す
def count_larger_than(S, threshold=0):
    assert isinstance(S, pd.core.series.Series)  # Sがシリーズであるかチェックを行っている
    return len([x for x in S if x > threshold])

関数の動作確認をしておきます。

In [179]:
s_test = pd.Series([1,3,5,6,8])
count_larger_than(s_test, 5)

2

In [180]:
df1

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0


`count_larger_than` 関数を各列に適用します。

In [181]:
df1.apply(count_larger_than, args=(10,))

sample1    2
sample2    3
sample3    3
dtype: int64

`count_larger_than` 関数を `axis=1` を指定して各行に適用します。

In [379]:
df1.apply(count_larger_than, args=(10,), axis=1)

geneA    0
geneB    2
geneC    3
geneD    3
dtype: int64

In [182]:
# シリーズを受け取ってシリーズまたはリストを返す関数
def my_cumproduct(S):
    L = []
    current = 1
    for x in S:
        current *= x
        L.append(current)
    return L

In [183]:
# 動作確認
my_cumproduct(pd.Series([10,20,30]))

[10, 200, 6000]

In [184]:
df1.apply(my_cumproduct)

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,60.0,60.0
geneC,1000.0,12000.0,18000.0
geneD,1000000.0,24000000.0,54000000.0


In [185]:
# 関数を作る:Seriesを引数として受け取り, Zスコア（平均値を引いた後、標準偏差で割る）に変換してSeriesとして返す
def zscore(S):
    mean = S.mean()  # 平均
    stdv = S.std()  # 標準偏差
    return (S - mean) / stdv

In [186]:
df1.apply(zscore)

,sample1,sample2,sample3
geneA,-0.572306,-0.571643,-0.572747
geneB,-0.553694,-0.554062,-0.553449
geneC,-0.367578,-0.367909,-0.367358
geneD,1.493578,1.493614,1.493554


`apply()` を使わずに計算することも可能です。

In [187]:
mean = df1.mean()
stdev = df1.std()
(df1-mean)/stdev

,sample1,sample2,sample3
geneA,-0.572306,-0.571643,-0.572747
geneB,-0.553694,-0.554062,-0.553449
geneC,-0.367578,-0.367909,-0.367358
geneD,1.493578,1.493614,1.493554


### 2-8. 行・列のループ処理
#### 2-8-1. データフレームをそのままループで回す

列名が順に取り出されます。

In [188]:
df1

,sample1,sample2,sample3
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0


In [189]:
for x in df1:
    print(x, type(x))

sample1 <class 'str'>
sample2 <class 'str'>
sample3 <class 'str'>


#### 2-8-2. 1行ずつ取り出す iterrows( )
`iterrows()`を使うと、各行のインデックスおよび行データがタプルとして取り出せます。  
行データの形式はシリーズになっています。

In [190]:
for index, row in df1.iterrows():
    print("INDEX =", index)
    print(list(row))
    print("-----")

INDEX = geneA
[1.0, 3.0, 2.0]
-----
INDEX = geneB
[10.0, 20.0, 30.0]
-----
INDEX = geneC
[100.0, 200.0, 300.0]
-----
INDEX = geneD
[1000.0, 2000.0, 3000.0]
-----


#### 2-8-3. 1列ずつ取り出す iteritems( )
`iteritems()`を使うと、各列の列名および列データがタプルとして取り出せる。  
列データの形式はシリーズになっています。

In [191]:
for col_name, col in df1.iteritems():
    print("Column name =", col_name)
    print(list(col))
    print("-----")    

Column name = sample1
[1.0, 10.0, 100.0, 1000.0]
-----
Column name = sample2
[3.0, 20.0, 200.0, 2000.0]
-----
Column name = sample3
[2.0, 30.0, 300.0, 3000.0]
-----


#### 2-8-4. forループを使う場合の注意点
N行x100列のテストデータの各要素を四捨五入した値に書き換える処理にかかる時間を, <br>
ループ処理, または `applymap()` を使った処理で比べてみましょう。<br>

テストデータ `test_df` を作ります。各データは　0~10　のランダムな小数です。

In [192]:
N=100
test_df = pd.DataFrame(np.random.rand(N, 100) * 10)

In [193]:
%%time
# データフレームをループでまわして、各要素を四捨五入した値に書き換える処理
# locを使う
for row_index, row in test_df.iterrows():
    for column_index in row.index:
        test_df.loc[row_index, column_index] = round(test_df.loc[row_index, column_index])
        # test_df.at[row_index, column_index] = round(test_df.at[row_index, column_index])

CPU times: user 1.69 s, sys: 20.4 ms, total: 1.71 s
Wall time: 1.69 s


これは, forループ自体が遅いわけではなく, ループの中で処理に時間がかかっています。<br>
`loc` を使った要素の参照・書き換えを行なっているのが遅くなる要因です。<br>
`loc` のかわりに `at` を使うと0.2秒程度まで改善されます。

In [194]:
N=100
test_df = pd.DataFrame(np.random.rand(N, 100) * 10)

In [195]:
%%time
# データフレームをループでまわして、各要素を四捨五入した値に書き換える処理
# atを使う
for row_index, row in test_df.iterrows():
    for column_index in row.index:       
        test_df.at[row_index, column_index] = round(test_df.at[row_index, column_index])

CPU times: user 125 ms, sys: 1.45 ms, total: 126 ms
Wall time: 125 ms


In [196]:
N=100
test_df = pd.DataFrame(np.random.rand(N, 100) * 10)

In [197]:
%%time
# ループを使わずに、applymapを使って各要素を四捨五入した値に書き換える処理
test_df = test_df.applymap(round)

CPU times: user 16 ms, sys: 1.25 ms, total: 17.3 ms
Wall time: 16.2 ms


forループの中で時間のかかる処理を行うとパフォーマンスが落ちます。<br>
一般的には, 
1. numpy や pandas の関数・メソッドを使用  
2. apply や applymap で行・列・データフレーム全体に関数を適用する  
3. for ループを回して処理する  

の順にパフォーマンスが低下します。

### 2-9. データフレームの結合
#### 2-9-1.  concat( )
2つ（またはそれ以上）のデータフレームを縦・横方向に連結します。共通するインデックスや列ラベルがあれば, それらを利用して結び付けられます。

In [198]:
df_A = pd.DataFrame([['A0','A1','A2'],
                     ['B0','B1','B2'],
                     ['C0','C1','C2'],
                     ['D0','D1','D2']],
                    columns=[0,1,2],
                    index=['A','B','C','D'])
df_B = pd.DataFrame([['E0','E1','E2','E3'],
                     ['F0','F1','F2','F3'],
                     ['G0','G1','G2','G3'],
                     ['H0','H1','H2','H3']],
                    columns=[0,1,2,3],
                    index=['E','F','G','H'])
df_C = pd.DataFrame([['A3','A4','A5'],
                     ['B3','B4','B5'],
                     ['C3','C4','C5'],
                     ['D3','D4','D5']],
                    columns=[3,4,5],
                    index=['A','B','C','D'])

In [199]:
df_A

,0,1,2
A,A0,A1,A2
B,B0,B1,B2
C,C0,C1,C2
D,D0,D1,D2


In [200]:
df_B

,0,1,2,3
E,E0,E1,E2,E3
F,F0,F1,F2,F3
G,G0,G1,G2,G3
H,H0,H1,H2,H3


In [201]:
df_C

,3,4,5
A,A3,A4,A5
B,B3,B4,B5
C,C3,C4,C5
D,D3,D4,D5


`df_A`, `df_B`を縦に連結します。

In [202]:
pd.concat([df_A, df_B])

,0,1,2,3
A,A0,A1,A2,NaN
B,B0,B1,B2,NaN
C,C0,C1,C2,NaN
D,D0,D1,D2,NaN
E,E0,E1,E2,E3
F,F0,F1,F2,F3
G,G0,G1,G2,G3
H,H0,H1,H2,H3


`df_A`, `df_C`を横方向に連結します。`axis=1`を指定し, 共通する行ラベル同士で連結します。

In [203]:
pd.concat([df_A, df_C], axis=1)

,0,1,2,3,4,5
A,A0,A1,A2,A3,A4,A5
B,B0,B1,B2,B3,B4,B5
C,C0,C1,C2,C3,C4,C5
D,D0,D1,D2,D3,D4,D5


縦方向に結合するなら`pandas.DataFrame` の `append( )` も簡単です。

In [204]:
df_A.append(df_B)

,0,1,2,3
A,A0,A1,A2,NaN
B,B0,B1,B2,NaN
C,C0,C1,C2,NaN
D,D0,D1,D2,NaN
E,E0,E1,E2,E3
F,F0,F1,F2,F3
G,G0,G1,G2,G3
H,H0,H1,H2,H3


#### 2-9-2. join( )
インデックス（行ラベル）を key として連結するのに便利です。

In [205]:
# テスト用データ
df_L = pd.DataFrame([1.2, 0.8, 2.3, 3.5, 2.2, 0.3],
                    index=['gene_1', 'gene_2', 'gene_3', 'gene_4', 'gene_5', 'gene_6'],
                    columns=['DE'])
df_R = pd.DataFrame(['50S ribosome-binding GTPase', 'Surface antigen', 'Elongation factor Tu GTP binding domain',
                     'Ring finger domain'],
                    index=['gene_1', 'gene_2', 'gene_4', 'gene_6'],
                    columns=['definition'])

In [206]:
df_L

,DE
gene_1,1.2
gene_2,0.8
gene_3,2.3
gene_4,3.5
gene_5,2.2
gene_6,0.3


In [207]:
df_R

,definition
gene_1,50S ribosome-binding GTPase
gene_2,Surface antigen
gene_4,Elongation factor Tu GTP binding domain
gene_6,Ring finger domain


In [208]:
df_L.join(df_R)

,DE,definition
gene_1,1.2,50S ribosome-binding GTPase
gene_2,0.8,Surface antigen
gene_3,2.3,NaN
gene_4,3.5,Elongation factor Tu GTP binding domain
gene_5,2.2,NaN
gene_6,0.3,Ring finger domain


上の場合, デフォルトでは `how="left"` が指定されているので, 左のデータフレーム（上の例ではdf_L）にあるものはすべて出力されます。

`how="inner"` を指定すると, 両方のデータフレームに存在するもののみが出力されます。

In [209]:
df_L.join(df_R, how='inner')

,DE,definition
gene_1,1.2,50S ribosome-binding GTPase
gene_2,0.8,Surface antigen
gene_4,3.5,Elongation factor Tu GTP binding domain
gene_6,0.3,Ring finger domain


`pd.concat()` でも同様の結果が得られます。

In [210]:
pd.concat([df_L, df_R], axis=1,sort=False)

,DE,definition
gene_1,1.2,50S ribosome-binding GTPase
gene_2,0.8,Surface antigen
gene_3,2.3,NaN
gene_4,3.5,Elongation factor Tu GTP binding domain
gene_5,2.2,NaN
gene_6,0.3,Ring finger domain


#### 2-9-3. merge( )
インデックス以外をkeyとして連結するのに便利です。多くのオプションがあるので詳細はpandasのヘルプを参照してください。

In [211]:
df_L = pd.DataFrame([['gene_1','PF00009'],
                     ['gene_2','PF01103'],
                     ['gene_3','PF01926'],
                     ['gene_4','PF01926'],
                     ['gene_5','PF13639'],
                     ['gene_6','PF02225']],
                    columns=['gene_id', 'PFAM_id',])
df_R = pd.DataFrame([['PF01926','50S ribosome-binding GTPase'],
                     ['PF01103','Surface antigen'],
                     ['PF00009','Elongation factor Tu GTP binding domain'],
                     ['PF13639', 'Ring finger domain']],
                    columns=['PFAM_id', 'definition'])

In [212]:
df_L

,gene_id,PFAM_id
0,gene_1,PF00009
1,gene_2,PF01103
2,gene_3,PF01926
3,gene_4,PF01926
4,gene_5,PF13639
5,gene_6,PF02225


In [213]:
df_R

,PFAM_id,definition
0,PF01926,50S ribosome-binding GTPase
1,PF01103,Surface antigen
2,PF00009,Elongation factor Tu GTP binding domain
3,PF13639,Ring finger domain


この例では, 共通する列ラベル PFAM_id をkeyとして結び付けられます。どの列をkeyにするかは、`on` , `left_on` , `right_on` などで指定可能です。インデックスの値をkeyとして使うことも可能です ( `left_index` , `right_index` )。<br>

デフォルトでは __innerjoin__(`how='inner'`)なので, 両方のデータに存在するもののみが結果として現れます。<br>
下の例では, 右側のデータフレーム（df_R）にない PF02225 は削除されます。

In [214]:
pd.merge(df_L, df_R)

,gene_id,PFAM_id,definition
0,gene_1,PF00009,Elongation factor Tu GTP binding domain
1,gene_2,PF01103,Surface antigen
2,gene_3,PF01926,50S ribosome-binding GTPase
3,gene_4,PF01926,50S ribosome-binding GTPase
4,gene_5,PF13639,Ring finger domain


__leftjoin__ (`how='left'`)にすると左側のデータフレーム（df_L）にあるものはすべて表示されます。

In [215]:
pd.merge(df_L, df_R, how='left')

,gene_id,PFAM_id,definition
0,gene_1,PF00009,Elongation factor Tu GTP binding domain
1,gene_2,PF01103,Surface antigen
2,gene_3,PF01926,50S ribosome-binding GTPase
3,gene_4,PF01926,50S ribosome-binding GTPase
4,gene_5,PF13639,Ring finger domain
5,gene_6,PF02225,NaN


以下の例では, df_A と df_C をインデックスをkeyとして結合します。<br>
`pd.concat([df_A, df_C], axis=1)`と同じ結果になります。

In [216]:
pd.merge(df_A, df_C, left_index=True, right_index=True)

,0,1,2,3,4,5
A,A0,A1,A2,A3,A4,A5
B,B0,B1,B2,B3,B4,B5
C,C0,C1,C2,C3,C4,C5
D,D0,D1,D2,D3,D4,D5


`pd.merge()` 以外に, 次のような書き方でも結合可能です。

In [217]:
df_L.merge(df_R)

,gene_id,PFAM_id,definition
0,gene_1,PF00009,Elongation factor Tu GTP binding domain
1,gene_2,PF01103,Surface antigen
2,gene_3,PF01926,50S ribosome-binding GTPase
3,gene_4,PF01926,50S ribosome-binding GTPase
4,gene_5,PF13639,Ring finger domain


### 2-10. その他の機能
__multiindex__<br>

複数の列をインデックスにすることができます。

In [39]:
df2

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
0,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
1,Volvox,carteri,v2.1,131.2,green algae,False
2,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
3,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True
4,Marchantia,polymorpha,v3.1,225.8,liverwort,False
5,Thalassiosira,pseudonana,V3.0,34.0,diatom,True


In [40]:
df2_multiindex = df2.set_index([df2.common_name, df2.generic_name])
df2_multiindex

generic_name         specific_name  \
common_name generic_name                                         
green algae Chlamydomonas  Chlamydomonas           reinhardtii   
            Volvox                Volvox               carteri   
            Ostreococcus    Ostreococcus           lucimarinus   
            Coccomyxa          Coccomyxa  subellipsoidea C-169   
liverwort   Marchantia        Marchantia            polymorpha   
diatom      Thalassiosira  Thalassiosira            pseudonana   

                          assemble_version  genome_size  common_name  \
common_name generic_name                                               
green algae Chlamydomonas             v5.5        111.1  green algae   
            Volvox                    v2.1        131.2  green algae   
            Ostreococcus              v2.0         13.2  green algae   
            Coccomyxa                 v2.0         49.0  green algae   
liverwort   Marchantia                v3.1        225.8    liverwort   
diatom      Thalassiosira             V3.0         34.0       diatom   

                           unicellular  
common_name generic_name                
green algae Chlamydomonas         True  
            Volvox               False  
            Ostreococcus          True  
            Coccomyxa             True  
liverwort   Marchantia           False  
diatom      Thalassiosira         True

In [41]:
df2_multiindex.loc['green algae']

,generic_name,specific_name,assemble_version,genome_size,common_name,unicellular
generic_name,,,,,,
Chlamydomonas,Chlamydomonas,reinhardtii,v5.5,111.1,green algae,True
Volvox,Volvox,carteri,v2.1,131.2,green algae,False
Ostreococcus,Ostreococcus,lucimarinus,v2.0,13.2,green algae,True
Coccomyxa,Coccomyxa,subellipsoidea C-169,v2.0,49.0,green algae,True


In [42]:
df2_multiindex.loc[('green algae', 'Chlamydomonas')]  # 複数インデックスの指定にはタプルを使用する

generic_name        Chlamydomonas
specific_name         reinhardtii
assemble_version             v5.5
genome_size                 111.1
common_name           green algae
unicellular                  True
Name: (green algae, Chlamydomonas), dtype: object

__groupby( )__<br>

データをグルーピングして扱うことができます。

In [218]:
df2.groupby('common_name').count()

,generic_name,specific_name,assemble_version,genome_size,unicellular
common_name,,,,,
diatom,1,1,1,1,1
green algae,4,4,4,4,4
liverwort,1,1,1,1,1


__pivot_table( )__<br>

カテゴリごとにグルーピングして計算するときに使います。

In [219]:
df2.pivot_table(index='common_name', values='genome_size', aggfunc='mean')  # グループごとの平均年齢を計算

,genome_size
common_name,
diatom,34.000
green algae,76.125
liverwort,225.800


### 2-11. データフレームの書き出し

ファイルに書き出した時に空欄にならないように, インデックスに名前をつけておきます(無くても良い)。

In [220]:
df1.index.name = 'INDEX' 
df1

,sample1,sample2,sample3
INDEX,,,
geneA,1.0,3.0,2.0
geneB,10.0,20.0,30.0
geneC,100.0,200.0,300.0
geneD,1000.0,2000.0,3000.0


`df1` をoutputというフォルダに `dataframe1.tsv` として書き出します。

In [ ]:
df1.to_csv('output/dataframe1.tsv', sep='\t', header=True, index=True)